# Guided Investigation - Anomaly Lookup

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0, azure-cli-profile 2.1.4<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebook on DSVM
     
__Data Source Required:__<br>
    -  Log Analytics tables 
    
### Description
Gain insights into the possible root cause of an alert by searching for related anomalies on the corresponding entities around the alert’s time. This notebook will provide valuable leads for an alert’s investigation, listing all suspicious increase in event counts or their properties around the time of the alert, and linking to the corresponding raw records in Log Analytics for the investigator to focus on and interpret.

<font>When you switch between Azure Notebooks Free Compute and Data Science Virtual Machine (DSVM), you may need to select Python version: please select Python 3.6 for Free Compute, and Python 3.6 - AzureML for DSVM.</font>

## Table of Contents

1. Initialize Azure Resource Management Clients
2. Looking up for anomaly entities

## 1. Initialize Azure Resource Management Clients

In [ ]:
# only run once
!pip install --upgrade Azure-Sentinel-Utilities
!pip install azure-cli-core

In [ ]:
# User Input and Save to Environmental store
import os
from SentinelWidgets import WidgetViewHelper
env_dir = %env
helper = WidgetViewHelper()

In [ ]:
# Enter Tenant Domain
helper.set_env(env_dir, 'tenant_domain')

In [ ]:
# Enter Azure Subscription Id
helper.set_env(env_dir, 'subscription_id')

In [ ]:
# Enter Azure Resource Group
helper.set_env(env_dir, 'resource_group')

In [ ]:
env_dir = %env
if 'tenant_domain' in env_dir:
    tenant_domain = env_dir['tenant_domain']
if 'subscription_id' in env_dir:
    subscription_id = env_dir['subscription_id']
if 'resource_group' in env_dir:
    resource_group = env_dir['resource_group']

In [ ]:
from azure.loganalytics import LogAnalyticsDataClient
from azure.loganalytics.models import QueryBody
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
import SentinelAzure
from SentinelAnomalyLookup import AnomalyFinder, AnomalyLookupViewHelper

from pandas.io.json import json_normalize
import sys
import timeit
import datetime as dt
import pandas as pd
import copy
from IPython.display import HTML

In [ ]:
# Authentication to Log Analytics
from azure.common.client_factory import get_client_from_cli_profile
from azure.common.credentials import get_azure_cli_credentials
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant $tenant_domain
la_client = get_client_from_cli_profile(LogAnalyticsManagementClient, subscription_id = subscription_id)
la = SentinelAzure.azure_loganalytics_helper.LogAnalyticsHelper(la_client)
creds, _ = get_azure_cli_credentials(resource="https://api.loganalytics.io")
la_data_client = LogAnalyticsDataClient(creds)

## 2. Looking up for anomaly entities

In [ ]:
# Select a workspace
selected_workspace = WidgetViewHelper.select_log_analytics_workspace(la)
display(selected_workspace)

In [ ]:
import ipywidgets as widgets
workspace_id = la.get_workspace_id(selected_workspace.value)
#DateTime format: 2019-07-15T07:05:20.000
q_timestamp = widgets.Text(value='2019-09-15',description='DateTime: ')
display(q_timestamp)
#Entity format: computer
q_entity = widgets.Text(value='computer',description='Entity for search: ')
display(q_entity)

In [ ]:
anomaly_lookup = AnomalyFinder(workspace_id, la_data_client)
selected_tables = WidgetViewHelper.select_multiple_tables(anomaly_lookup)
display(selected_tables)

In [ ]:
# This action may take a few minutes or more, please be patient.
start = timeit.default_timer()
anomalies, queries = anomaly_lookup.run(q_timestamp.value, q_entity.value, list(selected_tables.value))
display(anomalies)

if queries is not None:
    url = WidgetViewHelper.construct_url_for_log_analytics_logs(tenant_domain, subscription_id, resource_group, selected_workspace.value)
    WidgetViewHelper.display_html(WidgetViewHelper.copy_to_clipboard(url, queries, 'Add queries to clipboard and go to Log Analytics'))

print('==================')
print('Elapsed time: ', timeit.default_timer() - start, ' seconds')